# 0710012 何權祐 Final Project

In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib3
import re
from io import StringIO

%matplotlib inline

# Data Collection
Including Golden Globe, Oscar(winner & nominees)

data source:https://celebrityninjas.com/

celebrities:https://www.thefamouspeople.com/actors.php
            https://www.thefamouspeople.com/actress.php

### https://www.thefamouspeople.com/
爬蟲

In [42]:
data=pd.read_csv('actor_info.csv',sep='\t')
month_list=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
sign_list=['Capricorn','Aquarius','Pisces','Aries','Taurus','Gemini',"Cancer",'Leo','Virgo','Libra','Scorpio','Sagittarius']

for i in range(len(data.values)):
    if '.' in data['name'].values[i]:
        data['name'].values[i]=data['name'].values[i].replace('.','')
data

,name,b_date,b_month,b_year,sun_sign,b_country,job
0,Keanu Reeves,2,September,1964,Virgo,Lebanon,"television_actor,_film_actor,_actor,_film_prod..."
1,Leonardo DiCaprio,11,November,1974,Scorpio,United States,"actor,_producer"
2,Johnny Depp,9,June,1963,Gemini,United States,actor
3,Tom Cruise,3,July,1962,Cancer,United States,actor
4,Will Smith,25,September,1968,Libra,United States,actor
...,...,...,...,...,...,...,...
1531,Emily Ratajkowski,7,June,1991,Gemini,England,"model,_actress"
1532,Agnes Moorehead,6,December,1900,Sagittarius,United States,actress
1533,Annette Funicello,22,October,1942,Libra,United States,actress
1534,Kelly Rohrbach,21,January,1990,Aquarius,United States,actress


### https://celebrityninjas.com/
API

In [43]:
import itertools
data_2 = pd.read_json(r'test_fix.json')
for i in range(len(data_2)):
    if type(data_2['deathday'].values[i]) is float:
        data_2['deathday'].values[i]=0
    else:
        data_2['deathday'].values[i]=1
data_2

,name,gender,age,height,birthday,net_worth,nationality,occupation,deathday
0,Keanu Reeves,male,57.0,1.86,1964-09-02,360000000,ca,"[actor, voice_actor]",0
1,Leonardo DiCaprio,male,47.0,1.83,1974-11-11,260000000,us,"[actor, film_producer, television_producer]",0
2,Johnny Depp,male,58.0,1.78,1963-06-09,150000000,us,"[actor, film_producer, musician, film_director...",0
3,Tom Cruise,male,59.0,1.70,1962-07-03,600000000,us,"[actor, film producer, screenwriter]",0
4,Will Smith,male,53.0,1.88,1968-09-25,350000000,us,"[actor, film_producer, record_producer, televi...",0
...,...,...,...,...,...,...,...,...,...
1350,Molly Shannon,female,57.0,1.65,1964-09-16,7000000,us,"[actor, voice_actor, comedian]",0
1351,Emily Ratajkowski,female,30.0,1.71,1991-06-07,8000000,us,"[model, actor]",0
1352,Annette Funicello,female,70.0,1.60,1942-10-22,15000000,us,"[singer, actor]",1
1353,Kelly Rohrbach,NaN,NaN,NaN,NaN,2000000,NaN,NaN,0


# Preprocessing(Data Processing)

In [44]:
# count = 0
# for c in data_2:
#     if c.isna():
#         count+=1
# print(count)
# print(type(data_2.values[1433][3]))
data_2=data_2.dropna()
data_2=data_2.drop(columns='birthday')
data_2

,name,gender,age,height,net_worth,nationality,occupation,deathday
0,Keanu Reeves,male,57.0,1.86,360000000,ca,"[actor, voice_actor]",0
1,Leonardo DiCaprio,male,47.0,1.83,260000000,us,"[actor, film_producer, television_producer]",0
2,Johnny Depp,male,58.0,1.78,150000000,us,"[actor, film_producer, musician, film_director...",0
3,Tom Cruise,male,59.0,1.70,600000000,us,"[actor, film producer, screenwriter]",0
4,Will Smith,male,53.0,1.88,350000000,us,"[actor, film_producer, record_producer, televi...",0
...,...,...,...,...,...,...,...,...
1348,Christina Aguilera,female,41.0,1.57,160000000,us,"[record_producer, singer-songwriter, actor, tv...",0
1349,Barbra Streisand,female,79.0,1.65,400000000,us,"[film_director, actor, film_producer, singer-s...",0
1350,Molly Shannon,female,57.0,1.65,7000000,us,"[actor, voice_actor, comedian]",0
1351,Emily Ratajkowski,female,30.0,1.71,8000000,us,"[model, actor]",0


In [45]:
ind_list=[data.index[data['name'] == d].values[0] for d in data_2['name']]
# print(ind_list)
# print(data['name'].values[1192])
data_2.index=ind_list
data_2=data_2.drop(columns='name')

In [46]:
full_data=pd.concat([data,data_2],axis=1)
full_data=full_data.dropna()
country_list=np.asanyarray(pd.read_csv('country2num.txt',sep='\n',header=None))
country_list=[li[0].split(',') for li in country_list]
country_list=dict(country_list)
for i in range(len(full_data)):
    full_data['name'].values[i]=full_data['name'].values[i].lower()
    # print(country_list[full_data['b_country'][i]])
    full_data['b_country'].values[i]=country_list[full_data['b_country'].values[i]]
full_data=full_data.reset_index(drop=True)
full_data

,name,b_date,b_month,b_year,sun_sign,b_country,job,gender,age,height,net_worth,nationality,occupation,deathday
0,keanu reeves,2,September,1964,Virgo,27,"television_actor,_film_actor,_actor,_film_prod...",male,57.0,1.86,360000000.0,ca,"[actor, voice_actor]",0
1,leonardo dicaprio,11,November,1974,Scorpio,45,"actor,_producer",male,47.0,1.83,260000000.0,us,"[actor, film_producer, television_producer]",0
2,johnny depp,9,June,1963,Gemini,45,actor,male,58.0,1.78,150000000.0,us,"[actor, film_producer, musician, film_director...",0
3,tom cruise,3,July,1962,Cancer,45,actor,male,59.0,1.70,600000000.0,us,"[actor, film producer, screenwriter]",0
4,will smith,25,September,1968,Libra,45,actor,male,53.0,1.88,350000000.0,us,"[actor, film_producer, record_producer, televi...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1267,christina aguilera,18,December,1980,Sagittarius,45,singer-songwriter,female,41.0,1.57,160000000.0,us,"[record_producer, singer-songwriter, actor, tv...",0
1268,barbra streisand,24,April,1942,Taurus,45,singer_&amp;_actress,female,79.0,1.65,400000000.0,us,"[film_director, actor, film_producer, singer-s...",0
1269,molly shannon,16,September,1964,Virgo,45,comic_actress,female,57.0,1.65,7000000.0,us,"[actor, voice_actor, comedian]",0
1270,emily ratajkowski,7,June,1991,Gemini,14,"model,_actress",female,30.0,1.71,8000000.0,us,"[model, actor]",0


In [47]:
job_list=np.asanyarray(pd.read_csv('job_list.txt',sep='\n',header=None))
B_arr=[]
for i in range(len(full_data)):
    arr=[]
    for j in job_list:
        if str(j[0]) in str(full_data['job'].values[i]) or str(j[0]) in str(full_data['occupation'].values[i]):
            arr.append(1)
        else:
            arr.append(0)
    B_arr.append(arr)
df=pd.DataFrame(B_arr,columns=job_list.flatten())

In [48]:
from sklearn import preprocessing
full_data=pd.concat([full_data,df],axis=1)
name=np.asanyarray(full_data['name'].values)
full_data=full_data.drop(columns=['name','job','occupation'])
le = preprocessing.LabelEncoder()
pre_arr=['gender','sun_sign','nationality','b_month']
for col in pre_arr:
    full_data[col]=le.fit_transform(full_data[col])
full_data

,b_date,b_month,b_year,sun_sign,b_country,gender,age,height,net_worth,nationality,...,theatre_director,theatrical_producer,violinist,visual_artist,vj,voice_actress,voice_actor,waiting_staff,wrestler,writer
0,2,11,1964,11,27,1,57.0,1.86,360000000.0,3,...,0,0,0,0,0,0,1,0,0,0
1,11,9,1974,9,45,1,47.0,1.83,260000000.0,28,...,0,0,0,0,0,0,0,0,0,0
2,9,6,1963,4,45,1,58.0,1.78,150000000.0,28,...,0,0,0,0,0,0,1,0,0,1
3,3,5,1962,2,45,1,59.0,1.70,600000000.0,28,...,0,0,0,0,0,0,0,0,0,1
4,25,11,1968,6,45,1,53.0,1.88,350000000.0,28,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1267,18,2,1980,8,45,0,41.0,1.57,160000000.0,28,...,0,0,0,0,0,0,0,0,0,1
1268,24,0,1942,10,45,0,79.0,1.65,400000000.0,28,...,0,0,0,0,0,0,0,0,0,1
1269,16,11,1964,11,45,0,57.0,1.65,7000000.0,28,...,0,0,0,0,0,0,1,0,0,0
1270,7,6,1991,4,14,0,30.0,1.71,8000000.0,28,...,0,0,0,0,0,0,0,0,0,0


### get label

In [49]:
awarded=np.asanyarray(pd.read_csv(r'awarded.txt',sep='\n',header=None)).flatten()
awarded=[a.lower() for a in awarded]
Label=[np.int64(nm in awarded) for nm in name]

### shuffle

In [50]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    np.int64(full_data.values),Label,train_size=0.7,shuffle=True)

# Model、Result
## KNN

In [56]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from tabulate import tabulate
from sklearn.neighbors import KNeighborsClassifier
KNN_clt = KNeighborsClassifier(n_neighbors=15)
KNN_clt.fit(X_train,y_train)
KNN_predict = KNN_clt.predict(X_test)
print('Accuracy:')
print(round(KNN_clt.score(X_test,y_test),4)*100,'%')
print('\nPrecision:')
print(precision_score(np.uint32(y_test),np.uint32(KNN_predict),average=None,labels=[0,1]))
print('\nRecall(Sensitivity):')
print(recall_score(np.uint32(y_test),np.uint32(KNN_predict),average=None,labels=[0,1]))
print('\nConfusion_matrix:')
pd.DataFrame(confusion_matrix(np.uint32(y_test),np.uint32(KNN_predict),labels=[0,1]))

Accuracy:
71.47 %

Precision:
[0.74183007 0.60526316]

Recall(Sensitivity):
[0.88326848 0.368     ]

Confusion_matrix:


,0,1
0,227,30
1,79,46


## Random Forest

In [52]:
from sklearn.ensemble import RandomForestClassifier
RF_clt = RandomForestClassifier(n_estimators=25)
RF_clt.fit(X_train,y_train)
RF_predict = RF_clt.predict(X_test)
print('Accuracy:')
print(round(RF_clt.score(X_test,y_test),4)*100,'%')
print('\nPrecision:')
print(precision_score(np.uint32(y_test),np.uint32(RF_predict),average=None,labels=[0,1]))
print('\nRecall(Sensitivity):')
print(recall_score(np.uint32(y_test),np.uint32(RF_predict),average=None,labels=[0,1]))
print('\nConfusion_matrix:')
pd.DataFrame(confusion_matrix(np.uint32(y_test),np.uint32(RF_predict),labels=[0,1]))

Accuracy:
75.39 %

Precision:
[0.78006873 0.67032967]

Recall(Sensitivity):
[0.88326848 0.488     ]

Confusion_matrix:


,0,1
0,227,30
1,64,61


In [53]:
from xgboost import XGBClassifier

# 建立 XGBClassifier 模型
xgboostModel = XGBClassifier(n_estimators=1000, learning_rate= 0.01,use_label_encoder=False)
# 使用訓練資料訓練模型
xgboostModel.fit(X_train, y_train)
# 使用訓練資料預測分類
XG_predict = xgboostModel.predict(X_test)
print('Accuracy:')
print(round(xgboostModel.score(X_test,y_test),4)*100,'%')
print('\nPrecision:')
print(precision_score(np.uint32(y_test),np.uint32(XG_predict),average=None,labels=[0,1]))
print('\nRecall(Sensitivity):')
print(recall_score(np.uint32(y_test),np.uint32(XG_predict),average=None,labels=[0,1]))
print('\nConfusion_matrix:')
pd.DataFrame(confusion_matrix(np.uint32(y_test),np.uint32(XG_predict),labels=[0,1]))

[19:45:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy:
74.87 %

Precision:
[0.77288136 0.66666667]

Recall(Sensitivity):
[0.88715953 0.464     ]

Confusion_matrix:


,0,1
0,228,29
1,67,58
